In [19]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
plt.style.use('seaborn-whitegrid')
xl_01 = pd.ExcelFile('20190927_HeartSurvey_edited.xlsx')
survey = xl_01.parse("Sheet1")

xl_02 = pd.ExcelFile('Survey questions key.xlsx')
survey_key = xl_02.parse("Sheet1")



## Extract risk data to its own dataframe
risk = survey.iloc[:, 10:130:2].fillna(50).T ## All the actual questions, risk value

## Rename columns for risk dataframe
risk.columns = [i+1 for i in range(12)]
risk.index = [i+1 for i in range(60)]

## Extract waitlist decision data to its own dataframe
decision = survey.iloc[:, 11:130:2].T ## All the actual questions, risk value

## Rename waitlist decision columns to match the risk columns
decision.columns = [i+1 for i in range(12)]
decision.index = [i+1 for i in range(60)]

## Rename SRTR indices, scale SRTR risk to 1-100
survey_key.index = [i+1 for i in range(60)]
srtr_risk = pd.DataFrame(survey_key["SRTR Risk"])
#srtr_risk.index += 1
scaler = MinMaxScaler(feature_range=(0,100))
scaled_srtr_risk = scaler.fit_transform(srtr_risk)
scaled_srtr_risk = pd.DataFrame(scaled_srtr_risk, columns=['Scaled SRTR Risk'])
scaled_srtr_risk.index = [i+1 for i in range(60)]


decision.head()

,1,2,3,4,5,6,7,8,9,10,11,12
1,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
2,No,No,No,No,No,No,No,No,No,No,No,No
3,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
4,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
5,No,No,Yes,No,No,Yes,No,No,No,No,No,Yes


In [32]:
long_risk = pd.melt(risk, var_name='Observer', value_name='Observer Risk')
long_decision = pd.melt(decision, var_name='Observer', value_name='Observer Decision')
df = pd.DataFrame([long_risk['Observer'], long_risk['Observer Risk']])
df.loc[10]

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [10] of <class 'int'>

In [3]:
## Making a master "long" dataframe

# risk
# waitlist
# srtr_risk
# scaled_srtr_risk
##risks = risk.append(srtr_risk.T).append(scaled_srtr_risk.T)
# all_decisions = waitlist.append(srtr_risk.T).append(scaled_srtr_risk.T)
# all_decisions[all_decisions == 'green'] = 'yes'
# all_decisions[all_decisions == 'red'] = 'no'
observers = pd.DataFrame(np.arange(12)+1, columns=['Observer'], index=np.arange(12)+1)
risk_2 = risk.append(observers)
#risk_2 = pd.melt(risk)
# tidy_decisions = pd.melt(all_decisions.T, ["SRTR Risk"], var_name="Observer", value_name="Decision")
# tidy_data = pd.concat([tidy_risk.iloc[:720], tidy_decisions["Decision"].iloc[:720]], axis=1, sort=False)

observers


,Observer
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9
10,10
